<a href="https://colab.research.google.com/github/himanshira/Deep-Learning/blob/main/Classifying_Product_Reviews_with_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import csv
import nltk
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datafiniti/grammar-and-online-product-reviews")

print("Path to dataset files:", path)

100%|██████████| 8.95M/8.95M [00:00<00:00, 236MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/datafiniti/grammar-and-online-product-reviews/versions/1


In [ ]:
# Copy the folders from path to new directory

import shutil
import pathlib

data_dir = pathlib.Path("data")
if not data_dir.exists():
    shutil.copytree(path, data_dir)
    print("Dataset copied to data_dir")
else:
  print("'data' directory already exists. Skipping copy.")

Dataset copied to data_dir


#Build a CNN-LSTM hybrid model to classify the customer product reviews into good or bad.


In [ ]:
os.environ[ 'TF_ENABLE_ONEDNN_OPTS']='0'

In [ ]:
df = pd.read_csv('/content/data/GrammarandProductReviews.csv')
df['target'] = (df['reviews.rating'] < 4).astype(int)


In [ ]:
df.isnull().sum()

,0
id,0
brand,0
categories,0
dateAdded,0
dateUpdated,0
ean,31979
keys,0
manufacturer,141
manufacturerNumber,203
name,0


In [ ]:
df.dropna(subset=['reviews.text', 'reviews.rating'], inplace=True)
print(f"Dataset size after dropping NaNs: {len(df)}")

Dataset size after dropping NaNs: 71008


In [ ]:
df.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc,target
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11,0
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391,0
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391,0
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427,1
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71008 entries, 0 to 71043
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    71008 non-null  object 
 1   brand                 71008 non-null  object 
 2   categories            71008 non-null  object 
 3   dateAdded             71008 non-null  object 
 4   dateUpdated           71008 non-null  object 
 5   ean                   39031 non-null  object 
 6   keys                  71008 non-null  object 
 7   manufacturer          70867 non-null  object 
 8   manufacturerNumber    70806 non-null  object 
 9   name                  71008 non-null  object 
 10  reviews.date          70941 non-null  object 
 11  reviews.dateAdded     71008 non-null  object 
 12  reviews.dateSeen      71008 non-null  object 
 13  reviews.didPurchase   32156 non-null  object 
 14  reviews.doRecommend   60427 non-null  object 
 15  reviews.id            32

In [ ]:
X = df['reviews.text']
y = df['target']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Train size: 56806
Test size: 14202


In [ ]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 150

embedding_dim = 50

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

word_index = tokenizer.word_index

print(f"Unique tokens: {len(word_index)}")
print(f"Shape of X_train_padded: {X_train_padded.shape}")
print(f"Shape of X_test_padded: {X_test_padded.shape}")

Unique tokens: 26027
Shape of X_train_padded: (56806, 150)
Shape of X_test_padded: (14202, 150)


In [ ]:
# Build a CNN - LSTM hybrid model with the following layers:
# Input layer with input shape = MAX_SEQUENCE_LENGTH and dtype int32
# Embedding layer with input dimension = MAX_NB_WORDS, output dimension =50, and
# input length = MAX_SEQUENCE_LENGTH

VOCAB_SIZE = min(MAX_NB_WORDS, len(word_index)) + 1 # +1 for 0-padding or OOV token

from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model = tf.keras.Sequential([
    # Embedding layer: input_dim should be vocabulary size, not MAX_NB_WORDS itself
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE,
                              output_dim=embedding_dim,
                              input_length=MAX_SEQUENCE_LENGTH),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=5),
    tf.keras.layers.Dropout(0.2), # Use tf.keras.layers.Dropout for consistency
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=5),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid') # For binary classification, use 1 neuron with sigmoid
])
model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 150, 50)        │     1,000,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 146, 64)        │        16,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 25, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,069,747 (4.08 MB)

 Trainable params: 1,069,747 (4.08 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batch_size = 64
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=batch_size, validation_data=(X_test_padded, y_test), verbose=1)

Epoch 1/5
888/888 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8770 - loss: 0.3374 - val_accuracy: 0.9164 - val_loss: 0.2134
Epoch 2/5
888/888 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9298 - loss: 0.1879 - val_accuracy: 0.9230 - val_loss: 0.2020
Epoch 3/5
888/888 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9541 - loss: 0.1340 - val_accuracy: 0.9229 - val_loss: 0.2088
Epoch 4/5
888/888 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9682 - loss: 0.0959 - val_accuracy: 0.9202 - val_loss: 0.2209
Epoch 5/5
888/888 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9790 - loss: 0.0672 - val_accuracy: 0.9233 - val_loss: 0.2579


In [ ]:
train_acc = model.evaluate(X_train_padded, y_train, verbose=0)[1]
test_acc = model.evaluate(X_test_padded, y_test, verbose=0)[1]

In [ ]:
print(f"Train accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

Train accuracy: 0.9883815050125122
Test accuracy: 0.9232502579689026
